In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

from IPython.display import HTML
HTML('<iframe width="1100" height="619" src="https://www.youtube.com/embed/45Da3eqQKXQ" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

In [ ]:
import pandas as pd # Pandas for managing datasets - Для обработки и анализа данных
import numpy as np # Математические операциии и операции над тензорами

from matplotlib import pyplot # Matplotlib for additional customization - Визуализация данных 
import seaborn as sns # Seaborn for plotting and styling 

import datetime
from time import time
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.metrics import cohen_kappa_score, make_scorer
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.metrics import confusion_matrix
%load_ext autoreload
%autoreload 2

from xgboost import XGBClassifier, XGBRegressor, plot_importance
import xgboost as xgb
from sklearn.model_selection import KFold


In [ ]:
train = pd.read_csv('/kaggle/input/data-science-bowl-2019/train.csv')
test = pd.read_csv('/kaggle/input/data-science-bowl-2019/test.csv')
train_labels = pd.read_csv('/kaggle/input/data-science-bowl-2019/train_labels.csv')
specs = pd.read_csv('/kaggle/input/data-science-bowl-2019/specs.csv')

In [ ]:
train

In [ ]:
train_labels

In [ ]:
train_one_installation_id = train[(train.installation_id == "fffc0583")] # возьмем для наглядности один любой installation_id

In [ ]:
train_one_installation_id[(train_one_installation_id.game_session == 'c09b94eebfdf50a6')] # и для него возьмем любой game_session

In [ ]:
fig = plt.figure(figsize=(12,10))
ax1 = fig.add_subplot(211)
ax1 = sns.countplot(y="type", data=train, color="blue", order = train.type.value_counts().index)
plt.title("number of events by type")

ax2 = fig.add_subplot(212)
ax2 = sns.countplot(y="world", data=train, color="blue", order = train.world.value_counts().index)
plt.title("number of events by world")

plt.tight_layout(pad=0)
plt.show()


In [ ]:
plt.rcParams.update({'font.size': 12})

fig = plt.figure(figsize=(12,10))
se = train.title.value_counts().sort_values(ascending=True)
se.plot.barh()
plt.title("Event counts by title")
plt.xticks(rotation=0)
plt.show()

In [ ]:
def get_time(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['month'] = df['timestamp'].dt.month
    df['hour'] = df['timestamp'].dt.hour
    df['dayofweek'] = df['timestamp'].dt.dayofweek
    df['weekday_name'] = df['timestamp'].dt.weekday_name
    return df
result_with_time = get_time(train)
result_with_time_test = get_time(test)

In [ ]:
months = ["July", "Aug", "Sept", "Oct"]
plt.rcParams["figure.figsize"] = (8, 8)
game = train.groupby('month')['game_session'].agg('count')
fig, ax = plt.subplots()
ax.bar(months, game, color="green")
ax.set(title="Numer of Game Observations by Month")
ax.set(xlabel="Month", ylabel="game_session_count")

In [ ]:
months = ["July", "Aug", "Sept", "Oct"]
plt.rcParams["figure.figsize"] = (8, 8)
game = test.groupby('month')['game_session'].agg('count')
fig, ax = plt.subplots()
ax.bar(months, game, color="red")
ax.set(title="Numer of Game Observations by Month")
ax.set(xlabel="Month", ylabel="game_session_count")

In [ ]:
def dayofweek_vizualize(df):
    df.groupby('weekday_name')['game_session'] \
    .agg('count').T[['Monday','Tuesday','Wednesday',
                     'Thursday','Friday','Saturday',
                     'Sunday']].T.plot(figsize=(15, 3),
                                       title='Numer of Event Observations by Day of Week')
    return plt.show()
dayofweek_vizualize(result_with_time)

In [ ]:
dayofweek_vizualize(result_with_time_test)

In [ ]:
def conditions(df):
    time_period_conditions = [
        (df['hour'] >= 6) & (df['hour'] <= 9),
        (df['hour'] > 9) & (df['hour'] <= 17),
        (df['hour'] > 17) & (df['hour'] <= 22),
    ]
    time_period_choices = ['morning', 'day', 'evening']
    df['time_period'] = np.select(time_period_conditions, time_period_choices, default='night')
    
    week_days_conditions = [
        (df['dayofweek'] >= 1) & (df['dayofweek'] <= 5),
    ]
    week_days_choices = ['work_day']
    df['week_day'] = np.select(week_days_conditions, week_days_choices, default='weekend')
    return df
result = conditions(result_with_time)
result_test = conditions(result_with_time_test)

In [ ]:
def table_reduce(df):
    table1 = df.drop(columns=['event_id', 'event_code', 'event_data', 'timestamp', 'dayofweek', 'hour', 'month']) #удалили столбцы которые нам больше не понадобятся
    table2 = table1.groupby(['game_session', 'installation_id', 'title', 'type', 'world']).max().reset_index()  #сгруппировали по максимальным зн-ям  game_time и event_count, т е по одной игровой сессии 
    table3 = table2.drop(columns=['game_session', 'event_count', 'game_time']) # удалили столбцы которые нам больше не понадобятся 
    table_final = pd.get_dummies(table3, columns=['title', 'type', 'world','time_period', 'week_day']).groupby(['installation_id']).sum() #просуммировали все installation_id, для того чтобы посмотреть насколько часто каждый из признаков встречается в одном installation_id, и вносит свой вклад в оценку 
    return table_final
result_train = table_reduce(result)
result_test_2 = table_reduce(result_test)

In [ ]:
result_train

In [ ]:
small_labels = train_labels[['installation_id', 'accuracy_group']].groupby(['installation_id']).agg(lambda x:x.value_counts().index[0])
small_labels

In [ ]:
train_joined = result_train.join(small_labels).dropna()
train_joined

In [ ]:
target = train_joined['accuracy_group']
train_joined = train_joined.drop(['accuracy_group'], axis = 1)

In [ ]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.5, gamma=0.05, 
                             learning_rate=0.05, max_depth=2, 
                             min_child_weight=2, n_estimators=2000,
                             reg_alpha=0.4, reg_lambda=0.8,
                             subsample=0.75, silent=1,
                             random_state =7, nthread = -1)


# pars = {
#         'colsample_bytree': 0.8,                 
#         'learning_rate': 0.1,
#         'max_depth': 10,
#         'subsample': 1,
#         'objective':'multi:softprob',
#         'num_class':5,
#         'eval_metric':'mlogloss',
#         'min_child_weight':3,
#         'gamma':0.04,
#         'n_estimators':300
#     }

# kf = KFold(n_splits=5, shuffle=True, random_state=42)
# y_pre=np.zeros((len(table_final_test),5),dtype=float)
# final_test=xgb.DMatrix(table_final_test)


# for train_index, val_index in kf.split(train_joined):
#     train_X = train_joined.iloc[train_index]
#     val_X = train_joined.iloc[val_index]
#     train_y = target[train_index]
#     val_y = target[val_index]
#     xgb_train = xgb.DMatrix(train_X, train_y)
#     xgb_eval = xgb.DMatrix(val_X, val_y)
    
#     xgb_model = xgb.train(pars,
#                   xgb_train,
#                   num_boost_round=1000,
#                   evals=[(xgb_train, 'train'), (xgb_eval, 'val')],
#                   verbose_eval=False,
#                   early_stopping_rounds=100
#                  )
    
#     val_X=xgb.DMatrix(val_X)
#     pred_val=[np.argmax(x) for x in xgb_model.predict(val_X)]
    
#     print('choen_kappa_score :',cohen_kappa_score(pred_val,val_y,weights='quadratic'))
    
#     pred=xgb_model.predict(final_test)
#     y_pre+=pred
    
# pred = np.asarray([np.argmax(line) for line in y_pre])

In [ ]:
scores = cross_val_score(model_xgb, train_joined, target, cv = 5)
print(scores)
print(scores.mean())

In [ ]:
scores = [] 
X = result_train
kf = KFold(n_splits=2)
for train, test in kf.split(X):
    print("%s %s" % (train, test))    
model_xgb.fit(train_joined, target)

scores.append(model_xgb.score(train_joined, target))
print(scores)

In [ ]:
model_xgb.fit(X=train_joined, y=target)

In [ ]:
predictions = model_xgb.predict(result_test_2)

In [ ]:
df_groups = pd.DataFrame(predictions, columns = ['accuracy_group'])

groups = df_groups['accuracy_group'].values
groups[groups <= 1.2] = 0
groups[np.where(np.logical_and(groups > 1.2, groups <= 1.6))] = 1
groups[np.where(np.logical_and(groups > 1.6, groups <= 2.3))] = 2
groups[groups > 2.3] = 3

In [ ]:
result_test_2['accuracy_group'] = groups

In [ ]:
submission = result_test_2['accuracy_group'].astype(int).reset_index()
submission.to_csv('submission.csv', index=False)

submission

In [ ]:
result_test_2['accuracy_group'].value_counts()

In [ ]:
target.plot(kind='hist')

In [ ]:
submission['accuracy_group'].plot(kind='hist')